In [3]:
import sympy; import numpy as np;

class stable_matrix(sympy.Matrix):        
    def stab(self,k:int):
        M = self.copy()
        tmp = sympy.Matrix(np.zeros((M.shape[0]+k,M.shape[1]+k)))
        for i in range(min(M.shape[0]+k,M.shape[1]+k)):
            tmp[i,i] = 1
        tmp[:M.shape[0],:M.shape[1]]=M
        return tmp
    def __mul__(self,other):        
        if self.shape[1]==other.shape[0]:
            res = sympy.Matrix.__mul__(self,other)
        if self.shape[1]<other.shape[0]:
            k = other.shape[0]-self.shape[1]
            stabs = self.stab(k)
            res = sympy.Matrix.__mul__(stabs,other)
        if self.shape[1]>other.shape[0]:
            k = -other.shape[0]+self.shape[1]
            stabs = other.stab(k)
            res = sympy.Matrix.__mul__(self,stabs)
        return stable_matrix(res)
    def copy(self):
        return stable_matrix(sympy.Matrix(np.array(self,dtype=int)).copy())
    @property
    def pivot(self):
        from dataclasses import dataclass
        @dataclass
        class gauss_step_transformations:
            Left_inv : list
            Unit : list
            Left_tau : list
            Right_tau : list
            Right_inv : list         
            Left : list
            Right : list            
        @dataclass
        class transformation_tuple:
            from dataclasses import dataclass
            @dataclass
            class transformed_matrix:
                resA : list
            @dataclass
            class gauss_step_transformations:
                Left : list
                Left_inv : list
                Left_tau : list
                Unit : list
                Right_tau : list
                Right : list
                Right_inv : list        
            resA : transformed_matrix
            trsf : gauss_step_transformations
        if len(self) == 0:
            return self
        min_v = min( abs(v) for v in self if v!=0 )
        k,l = -1,-1
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                if abs(self[i,j])==min_v:
                    k,l=i,j
                    break
            if (k,l) != (-1,-1):
                break
        print(k,l,min_v)
        tmp = stable_matrix(self.copy())
        TL,U,TR = stable_matrix(np.eye(self.shape[0],dtype=int)),\
                  stable_matrix(np.eye(1,dtype=int)),\
                  stable_matrix(np.eye(self.shape[1],dtype=int))
        if k>0:
            print(f"switch {k+1}th and first row")
            tmp = stable_matrix.tij(0,k)*tmp
            TL = TL*stable_matrix.tij(0,k)
        if l>0:
            print(f"switch {l+1}th and first column")
            tmp = tmp*stable_matrix.tij(0,l)
            TR = stable_matrix.tij(0,l)*TR
        if tmp[0,0]<0:
            tmp = tmp*stable_matrix.ui(0,-1)
            U = -1
        assert tmp[0,0]>0
        # now we've arranged for tmp[0,0] to be a norm minimal
        # non-zero matrix entry
        res = transformation_tuple(resA = tmp,trsf = gauss_step_transformations(Left=np.eye(1,dtype=int),Left_inv=np.eye(1,dtype=int),Left_tau=TL,Unit=U,Right_tau=TR,Right=np.eye(1,dtype=int),Right_inv=np.eye(1,dtype=int)))
        return res
    @property
    def one_gauss(self):
        from dataclasses import dataclass
        @dataclass
        class gauss_step_transformations:
            Left : list
            Left_inv : list
            Left_tau : list
            Unit : list
            Right_tau : list
            Right : list
            Right_inv : list         
        @dataclass
        class transformation_tuple:
            from dataclasses import dataclass
            @dataclass
            class transformed_matrix:
                resA : list
            @dataclass
            class gauss_step_transformations:
                Left : list
                Left_inv : list
                Left_tau : list
                Unit : list
                Right_tau : list
                Right : list
                Right_inv : list        
            resA : transformed_matrix
            trsf : gauss_step_transformations        
        eij = stable_matrix.eij
        pivot_trsf = self.pivot
        tmp = pivot_trsf.resA.copy()
        L = stable_matrix(np.eye(self.shape[0],dtype=int))
        R = stable_matrix(np.eye(self.shape[1],dtype=int))
        Li = stable_matrix(np.eye(self.shape[0],dtype=int))
        Ri = stable_matrix(np.eye(self.shape[1],dtype=int))        
        pivot = tmp[0,0]
        assert tmp[0,0]>0
        for row_ix in range(1,tmp.shape[0]):
            val = tmp[row_ix,0] - (tmp[row_ix,0] % pivot)
            tmp = eij(row_ix,0,-val//pivot)*tmp
            L = L*eij(row_ix,0,val//pivot)
            Li = eij(row_ix,0,-val//pivot)*Li
        for col_ix in range(1,tmp.shape[1]):
            val = tmp[0,col_ix] - (tmp[0,col_ix] % pivot)
            tmp = tmp*eij(0,col_ix,-val//pivot)
            R = eij(0,col_ix,val//pivot)*R
            Ri = Ri*eij(0,col_ix,-val//pivot)
        pivot_trsf.resA = tmp
        pivot_trsf.trsf.Left = L
        pivot_trsf.trsf.Left_inv = Li
        pivot_trsf.trsf.Right = R
        pivot_trsf.trsf.Right_inv = Ri
        return pivot_trsf
    @property
    def gauss(self):
        def matrix_pivot_splittable(A):
            row_max,col_max=0,0
            if A.shape[0]>1 and A.shape[1]>1:
                col_max = abs(np.array(A[1:,0])).max()            
                row_max = abs(np.array(A[0,1:])).max()
            mx = max(col_max,row_max)
            return mx == 0
        transformations = list()
        divisors = list()
        step = self
        while True:
            step = step.one_gauss
            tmp = step.resA
            transformations = transformations + [step.trsf]
            if matrix_pivot_splittable(tmp):
                divisors = divisors + [tmp[0,0]]
                tmp = tmp[1:,1:]
            step = tmp
            if min(step.shape)==0:
                break
        return divisors, transformations
    @classmethod
    def eij(cls,i,j,k=1):
        assert i>=0
        assert j>=0
        assert i!=j
        n = max(i,j)+1
        M = sympy.eye(n)
        M[i,j]=k
        return stable_matrix(M)
    @classmethod
    def tij(cls,i,j):
        assert i>=0
        assert j>=0
        assert i!=j
        n = max(i,j)+1
        M = sympy.zeros(n,n)
        for k in range(n):
            if k not in [i,j]:
                M[k,k]=1
        M[i,j]=1; M[j,i]=1
        assert (M**2==sympy.eye(n))
        return stable_matrix(M)
    @classmethod
    def ui(cls,i,u=-1):
        assert i>=0
        n = i+1
        M = sympy.eye(n)
        M[i,i]=u
        return stable_matrix(M)

eij = stable_matrix.eij
tij = stable_matrix.tij
ui = stable_matrix.ui

A = stable_matrix(np.random.randint(-5,15,(5,7)))
A

Matrix([
[ 1,  5, -2, -2,  9,  6, -3],
[13, 14,  7, 10, -4,  5,  9],
[12,  0,  6, 12, -3,  7, -2],
[ 4, 14,  1, 11, 11, -4,  3],
[-4,  9, 12, -5, -3,  1, 10]])

In [4]:
res = A.gauss
res

0 0 1
3 5 2
switch 4th and first row
switch 6th and first column
0 2 1
switch 3th and first column
1 0 39
switch 2th and first row
0 1 1
switch 2th and first column
0 2 45
switch 3th and first column
0 2 11
switch 3th and first column
0 2 1
switch 3th and first column
0 2 10701
switch 3th and first column


([1, 1, 1, 1, 10701],
 [stable_matrix.pivot.<locals>.gauss_step_transformations(Left_inv=Matrix([
  [  1, 0, 0, 0, 0],
  [-13, 1, 0, 0, 0],
  [-12, 0, 1, 0, 0],
  [ -4, 0, 0, 1, 0],
  [  4, 0, 0, 0, 1]]), Unit=Matrix([[1]]), Left_tau=Matrix([
  [1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0],
  [0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1]]), Right_tau=Matrix([
  [1, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 1]]), Right_inv=Matrix([
  [1, -5, 2, 2, -9, -6, 3],
  [0,  1, 0, 0,  0,  0, 0],
  [0,  0, 1, 0,  0,  0, 0],
  [0,  0, 0, 1,  0,  0, 0],
  [0,  0, 0, 0,  1,  0, 0],
  [0,  0, 0, 0,  0,  1, 0],
  [0,  0, 0, 0,  0,  0, 1]]), Left=Matrix([
  [ 1, 0, 0, 0, 0],
  [13, 1, 0, 0, 0],
  [12, 0, 1, 0, 0],
  [ 4, 0, 0, 1, 0],
  [-4, 0, 0, 0, 1]]), Right=Matrix([
  [1, 5, -2, -2, 9, 6, -3],
  [0, 1,  0,  0, 0, 0,  0],
  [0, 0,  1,  0, 0, 0,  0],
  [0, 0,  0,  1, 0, 0,  0],
  [0